In [1]:
from elasticsearch import Elasticsearch

In [5]:
es = Elasticsearch('https://localhost:9200',verify_certs=False,basic_auth=("elastic","wHJp0zW7Qo9R-I6Uaeg6"))
es.ping()


d:\Projects\ElasticSearch\venv\Lib\site-packages\elasticsearch\_sync\client\__init__.py:403: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
d:\Projects\ElasticSearch\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True

In [7]:
import pandas as pd

df = pd.read_csv("myntra_products_catalog.csv").loc[:1000]
df.tail()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
996,10029377,Geox Men Burgundy Suede Loafers,Geox,Men,5999,5,"A pair of round-toe burgundy loafers, has regu...",Burgundy
997,10030893,Tulsattva Women Brown & Red Solid Kurta with P...,Tulsattva,Women,1347,5,Brown and red solid kurta with palazzos Brown ...,Red
998,10032495,HERE&NOW Women Teal Green Solid Crop Sweatshirt,HERE&NOW,Women,1299,5,"Teal green solid crop sweatshirt, has a mandar...",Green
999,10025889,ExclusiveLane Brown Set of Single Geometric Sq...,ExclusiveLane,Unisex,499,5,Set content: Single Cushion CoversColour: Come...,Brown
1000,10018839,Raymond Men Beige Nancy Regular Fit Self Desig...,Raymond,Men,1484,5,"Beige self-design formal shirt, has a spread c...",Beige


In [8]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           943
                                                                                   True             58
Name: count, dtype: int64

In [9]:
df.fillna("None",inplace=True)

In [10]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

d:\Projects\ElasticSearch\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Projects\ElasticSearch\venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\moham\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activ

In [11]:
df["DescriptionVector"] = df['Description'].apply(lambda x : model.encode(x))

In [12]:
from indexMapping import indexMapping

es.indices.create(index = "all_products" , mappings=indexMapping)

d:\Projects\ElasticSearch\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products'})

In [14]:
record_list = df.to_dict("records")

In [15]:
for record in record_list:
    try:
        es.index(index="all_products",document=record,id=record["ProductID"])
    except Exception as e:
        print(e)

d:\Projects\ElasticSearch\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [16]:
es.count(index="all_products")

d:\Projects\ElasticSearch\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'count': 1001, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [20]:
input_keyword = "Blue shoes"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 500
}

In [21]:
result = es.knn_search(
    index="all_products",
    knn = query,
    source=["ProductName","Description"]
)

C:\Users\moham\AppData\Local\Temp\ipykernel_7812\1976251380.py:1: GeneralAvailabilityWarning: This API is in technical preview and may be changed or removed in a future release. Elastic will work to fix any issues, but features in technical preview are not subject to the support SLA of official GA features.
  result = es.knn_search(
d:\Projects\ElasticSearch\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\moham\AppData\Local\Temp\ipykernel_7812\1976251380.py:1: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  result = es.knn_search(


In [22]:
result["hits"]["hits"]

[{'_index': 'all_products',
  '_id': '10017133',
  '_score': 0.62217253,
  '_source': {'ProductName': 'Carrera Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailSynthetic upperCushioned footbedTextured and patterned outsoleWarranty: 1 monthWarranty provided by brand/manufacturer'}},
 {'_index': 'all_products',
  '_id': '10018013',
  '_score': 0.6175091,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}}]